# Faster R-CNN

## Automated Library Installer in Python

This script automates the installation of Python libraries listed in a JSON file. It checks whether each library is already installed, installs any missing dependencies using pip, and reports the success or failure of each installation. With built-in error handling and a set of commonly used libraries preloaded, the script streamlines dependency management for Python projects.

## Import the necessary libraries.

## Decide whether to use CUDA (GPU), MPS or CPU

This code detects and selects the most suitable device for running deep learning models, prioritizing hardware acceleration through CUDA (GPU) or MPS (Metal Performance Shaders). If no supported acceleration is available, it automatically falls back to the CPU and informs the user.

## Training process for the Faster R-CNN object detection model

### Declare the possible classes of the dataset